# Assignment:  Segmenting and Clustering Neighborhoods in Toronto

#### Part One: Scrape the Wikipedia page,transform the data into a pandas dataframe

In [17]:
#For data manipulation and analysis(Dataframe)
import pandas as pd

#For parsing XML and HTML documents
!pip install requests
import requests

!pip install lxml
from lxml import html

#Parsing HTML
#url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050." old version
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

#Store the contents of the website
data = html.fromstring(page.content)

#Store table data
#XPath is a way of locating information in structured documents such as HTML or XML documents
tr_elements = data.xpath('//tr')

col=[]
ind=0

#store each first element and an empty list for each row
for i in tr_elements[0]:
    ind= ind + 1
    name = i.text_content()
    col.append((name,[]))
print(col)

#Since the first row is header, data is stored from the second row
for i in range(1,len(tr_elements)):
    T=tr_elements[i]
    
    #If row is not of size 3, then tr data is not from our table 
    if len(T)!=3:
        break
    #i is the index of our column
    ind=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        temp=t.text_content() 
        #Check if row is empty
        if ind>0:
        #Convert any numerical value to integers
            try:
                temp=int(temp)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[ind][1].append(temp)
        #Increment i for the next column
        ind = ind + 1
        
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

[('Postal Code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]


In [18]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [19]:
#Replace bogus \n from data

cols = ["Postal Code\n", "Borough\n", "Neighborhood\n"]

for col in cols:
    df[col] = df[col].map(lambda x: str(x).lstrip('\n').rstrip('\n')).astype(str)

In [20]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [34]:
df.rename(columns={"Postal Code\n": "Postal Code", "Neighborhood\n" : "Neighborhood", "Borough\n":"Borough" } ,inplace = True)

In [35]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [36]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

df = df.reset_index(drop = True)

df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [37]:
df.shape

(104, 3)